In [1]:
import cv2
import numpy as np

# Load the two images
image1 = cv2.imread('image1.jpg')
image2 = cv2.imread('image2.jpg')

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Find keypoints and descriptors with SIFT
keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)  # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params, search_params)

# Matching descriptor using KNN algorithm
matches = flann.knnMatch(descriptors1, descriptors2, k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]

# Ratio test as per Lowe's paper
good = []
for i, (m, n) in enumerate(matches):
    if m.distance < 0.7 * n.distance:
        matchesMask[i] = [1,0]
        good.append(m)

if len(good) >= 4:
    src_pts = np.float32([ keypoints1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ keypoints2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    # Compute Homography
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    print("Homography matrix is:")
    print(H)
else:
    print("Not enough matches are found - {}/{}".format(len(good), 4))

# Optionally draw matches.
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv2.DrawMatchesFlags_DEFAULT)

img_matches = cv2.drawMatchesKnn(image1, keypoints1, image2, keypoints2, matches, None, **draw_params)

cv2.imshow("Matches", img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()


Homography matrix is:
[[ 9.71198937e-01  4.90689520e-02  8.97001998e+01]
 [-1.94642845e-02  1.02335454e+00  1.19517888e+00]
 [-1.11133069e-04  1.07345237e-04  1.00000000e+00]]
